In [1]:
import glob
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
def extract_features(file_name):
    X, sample_rate = librosa.load(file_name)
    stft = np.abs(librosa.stft(X))
    mfccs = np.array(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=8).T)
    chroma = np.array(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T)
    mel = np.array(librosa.feature.melspectrogram(X, sr=sample_rate).T)
    contrast = np.array(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T)
    tonnetz = np.array(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T)
    return mfccs,chroma,mel,contrast,tonnetz

In [3]:
def parse_audio_files(parent_dir,sub_dirs,file_ext='*.wav'):
    ignored = 0
    features, labels, name = np.empty((0,161)), np.empty(0), np.empty(0)
    for label, sub_dir in enumerate(sub_dirs):
        print(sub_dir)
        for fn in glob.glob(os.path.join(parent_dir, sub_dir, file_ext)):
            try:
                mfccs, chroma, mel, contrast, tonnetz = extract_features(fn)
                ext_features = np.hstack([mfccs, chroma, mel, contrast, tonnetz])
                features = np.vstack([features,ext_features])
                l = [fn.split('-')[1]] * (mfccs.shape[0])
                labels = np.append(labels, l)
            except (KeyboardInterrupt, SystemExit):
                raise
            except:
                ignored += 1
    print ("Ignored files: ", ignored)
    return np.array(features), np.array(labels, dtype = np.int)

## load sounds and extract features and labels

In [4]:
parent_dir = 'C:/Users/rano4/Desktop/5thyear/graduationproject/Project/UrbanSound8K/audio'

sub_dirs = ['fold1', 'fold2', 'fold3', 'fold4', 'fold5', 'fold6', 'fold7', 'fold8', 'fold9', 'fold10']
#sub_dirs = ['fold1']

try:
    labels = np.load('labels.npy')
    features = np.load('features.npy')
    print("Features and labels found!")
except:
    print("Extracting features...")
    features, labels = parse_audio_files(parent_dir,sub_dirs)
    with open('features.npy', 'wb') as f1:
            np.save(f1,features)
    with open('labels.npy', 'wb') as f2:
            np.save(f2, labels)

Features and labels found!


In [5]:
print("Splitting Data!")
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=0)

Splitting Data!


## Normalization

In [6]:
print("normalization data")
sc = StandardScaler()
sc.fit(X_train)

normalization data


StandardScaler(copy=True, with_mean=True, with_std=True)

In [7]:
with open("fit_params.npy", "wb") as f3:
    np.save(f3, X_train)
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)

## fitting and Training data

In [8]:
from sklearn.neighbors import KNeighborsClassifier

In [9]:
clf = KNeighborsClassifier(n_neighbors=3)
clf.fit(X_train, y_train) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [ ]:
print('score (training): {:.3f}'
     .format(clf.score(X_train, y_train)))
print('score (test): {:.3f}'
     .format(clf.score(X_test, y_test)))

In [ ]:
#default 
score (training): 0.686
score (test): 0.526
    
# 5 
score (training): 0.686
score (test): 0.526
    
#3
score (training): 0.686
score (test): 0.526
    
#50
score (training): 0.510
score (test): 0.489